In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [7]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [8]:
wiki.name

'wikipedia'

In [11]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Import your Ollama wrapper
from langchain_community.embeddings import OllamaEmbeddings

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

# Use OllamaEmbeddings with the "nomic-embed-text" model
vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020C0E0F3B60>, search_kwargs={})

In [12]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [13]:
retriever_tool.name

'langsmith_search'

In [14]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [15]:
tools=[wiki,arxiv,retriever_tool]

In [16]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\manoj\\Desktop\\chatbot\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000020C27DA3560>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_com

In [31]:
from langchain_community.llms import Ollama

# Instantiate an Ollama LLM instead of OpenAI
llm = Ollama(model="gemma3")


In [32]:
from langchain import hub
from langchain_community.llms import Ollama

# Pull the agent prompt from the hub (this works for Ollama too)
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

# Setup Ollama LLM
llm = Ollama(model="gemma3")   # Replace "llama3" with your Ollama model

# When using with an agent, supply 'llm', and 'prompt' as needed


c:\Users\manoj\Desktop\chatbot\venv\Lib\site-packages\langsmith\client.py:288: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [33]:
from langchain_community.llms import Ollama
llm = Ollama(model="gemma3")  # or your ollama-supported model

from langchain import hub

# Pull a prompt that doesn't require chat_history
prompt = hub.pull("hwchase17/react-chat")  # Ensure this prompt matches your needs

# Use the 'create_react_agent' NOT 'create_openai_tools_agent'
from langchain.agents import create_react_agent

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor


c:\Users\manoj\Desktop\chatbot\venv\Lib\site-packages\langsmith\client.py:288: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'OllamaEmbeddings\'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020C

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [ ]:
agent_executor.invoke({
    "input": "Tell me about Langsmith",
    "chat_history": []  # Provide an empty list or previous chat history
})



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: langsmith_search
Action Input: "LangSmith"LangSmith is framework agnostic — you can use it with or without LangChain’s open source frameworks langchain and langgraph.

Get started with LangSmith - Docs by LangChainOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KAsk AIForumForumSearch...NavigationGet started with LangSmithGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationForumGet started

In [2]:
pip install arxiv

  Using cached arxiv-2.2.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached feedparser-6.0.11-py3-none-any.whl.metadata (2.4 kB)
  Using cached sgmllib3k-1.0.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6105 sha256=fe3aa7f5db82b1ba17ca6bb9b667dfa2c8e682a6f9532f89ee3436b54e7e1ff7
  Stored in directory: c:\users\manoj\appdata\local\pip\cache\wheels\3d\4d\ef\37cdccc18d6fd7e0dd7817dcdf9146d4d6789c32a227a28134
Successfully built sgmllib3k

   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- ------------------------

  DEPRECATION: Building 'sgmllib3k' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'sgmllib3k'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [6]:
pip install wikipedia

  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11785 sha256=94b355c5682e9c3520adc3d2f62465e7c8dc6207e826b5015ebbec8ead617e65
  Stored in directory: c:\users\manoj\appdata\local\pip\cache\wheels\79\1d\c8\b64e19423cc5a2a339450ea5d145e7c8eb3d4aa2b150cde33b
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'wikipedia' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wikipedia'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [2]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

ImportError: Could not import wikipedia python package. Please install it with `pip install wikipedia`.

In [3]:
pip intall wikipedia


Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "intall" - maybe you meant "install"



In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable